In [48]:
import numpy as np
import pandas as pd
from pprint import pprint
from movie import *
import pickle
from multiprocessing import Pool
from datetime import datetime
from pprint import pprint
import math

from sklearn.feature_selection import VarianceThreshold
import sklearn.datasets as datasets
from sklearn.feature_selection import SelectPercentile,f_classif, f_regression,SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

In [12]:
with open('name_url.pkl','rb') as f:
    name_info=pickle.load(f)

print(name_info['Fahrenheit 9/11(2004)'])

{'Release Date': datetime.datetime(2004, 6, 23, 0, 0), 'url': 'https://www.the-numbers.com/movie/Fahrenheit-9-11', 'Production Budget': 6000000, 'Domestic Gross': 119114517, 'Worldwide Gross': 221148959}


## Data preprocessing


In [2]:
def get_roles():
    with open('movie_info.pkl', 'rb') as f:
        movie_info=pickle.load(f)
    
    role_set=set()
    for name,info in movie_info.items():
        if 'production' in info['cast'].keys():
            for person in info['cast']['production']:
                role_set.add(person['role'])
    
    return list(role_set)

In [34]:
print(len(get_roles()))

1965


In [3]:
with open('name_url.pkl','rb') as f:
    name_dic=pickle.load(f)

print(name_dic['Deep Throat(1972)'])

{'Release Date': datetime.datetime(1972, 6, 30, 0, 0), 'url': 'https://www.the-numbers.com/movie/Deep-Throat', 'Production Budget': 25000, 'Domestic Gross': 45000000, 'Worldwide Gross': 45000000}


In [68]:
with open('movie_info.pkl','rb') as f:
    movie_info=pickle.load(f)
    
def get_person_info_helper(name, movie_info=movie_info, movie_budget=movie_budget):
    
    #print(name)
    person_name_info=dict()
    #cast_url=value+"#tab=cast-and-crew"
    cast_dic=movie_info[name]['cast']
    cast_list=['leading_members', 'supporting_cast', 'cameos', 'uncategorized', 'production']
    if 'Domestic Releases' in movie_info[name]['summary'].keys() and min(movie_info[name]['summary']['Domestic Releases'].values())<datetime.now():
        for cate in cast_list:
            if cate in cast_dic.keys():
                for person in cast_dic[cate]:
                    person_name=person['name']
                    if person_name in person_name_info.keys():
                        person_name_info[person_name]=set(list(person_name_info[person_name])+[name])
                    else:
                        person_name_info[person_name]=set([name])
                    
    return person_name_info

In [66]:
get_person_info_helper('Black Panther(2018)')

{'Andy Gill': {'Black Panther(2018)'},
 'Andy Serkis': {'Black Panther(2018)'},
 'Angela Bassett': {'Black Panther(2018)'},
 'Atandwa Kani': {'Black Panther(2018)'},
 'Chadwick Boseman': {'Black Panther(2018)'},
 'Claudia Castello': {'Black Panther(2018)'},
 'Connie Chuene': {'Black Panther(2018)'},
 'Danai Gurira': {'Black Panther(2018)'},
 'Daniel Kaluuya': {'Black Panther(2018)'},
 'Dave Jordan': {'Black Panther(2018)'},
 'David J. Grant': {'Black Panther(2018)'},
 'Florence Kasumba': {'Black Panther(2018)'},
 'Forest Whitaker': {'Black Panther(2018)'},
 'Geoffrey Baumann': {'Black Panther(2018)'},
 'Hannah Beachler': {'Black Panther(2018)'},
 'Isaach de Bankole': {'Black Panther(2018)'},
 'Jack Kirby': {'Black Panther(2018)'},
 'Jeffrey Chernov': {'Black Panther(2018)'},
 'Joe Robert Cole': {'Black Panther(2018)'},
 'John Kani': {'Black Panther(2018)'},
 'Kevin Feige': {'Black Panther(2018)'},
 'Letitia Wright': {'Black Panther(2018)'},
 'Louis D’Esposito': {'Black Panther(2018)'},

In [88]:
def get_all_person_urls():
    with open('movie_info.pkl', 'rb') as f:
        movie_info=pickle.load(f)
    
    with open('name_url.pkl','rb') as f:
        movie_budget=pickle.load(f)
    
    total_dic=dict()
    keys=list(movie_info.keys())
    pool = Pool(processes=10)
    pool_outputs = pool.map(get_person_info_helper,keys)
    pool.close()
    pool.join()
    
    for item in pool_outputs:
        for key, value in item.items():
            total_dic[key.strip()]=set(list(total_dic.get(key.strip(),[]))+list(value))
    
    final_dic=dict()
    for key, value in total_dic.items():
        total_sum=0
        total_count=0
        for name in value:
            if movie_budget[name]['Worldwide Gross']>0:
                total_sum+=movie_budget[name]['Worldwide Gross']
                total_count+=1
        
        final_dic[key]=math.log(total_sum/(total_count+0.1)+1)
    
    with open('person_info.pkl','wb') as f:
        pickle.dump(final_dic, f)
        
    return

In [89]:
get_all_person_urls()

In [90]:
import operator
with open('person_info.pkl', 'rb') as f:
    person_info=pickle.load(f)
    
sorted_person=sorted(person_info.items(), key=lambda d: d[1])
print(sorted_person[-30:-1])
print(sorted_person[:30])

[('Jim McGrath', 21.350012024183115), ('Jessica Sherman', 21.350012024183115), ('Rachel Dill', 21.350012024183115), ('Janet Lewin', 21.350012024183115), ("Luke O'Byrne", 21.350012024183115), ('James Clyne', 21.350012024183115), ('Kevin Jenkins', 21.350012024183115), ('Oliver Roberts', 21.350012024183115), ('Aaron E. Schneider', 21.420137163394873), ('Sean Anthony Moran', 21.65181529829283), ('Jason Whyte', 21.65181529829283), ('James Pitt', 21.65181529829283), ('Peter Dillon', 21.65181529829283), ('Kelson Henderson', 21.65181529829283), ('David Van Horn', 21.65181529829283), ('Sonia Yee', 21.65181529829283), ('Jahnel Curfman', 21.65181529829283), ('Kyla Warren', 21.65181529829283), ('Lucy Briant', 21.65181529829283), ('Nathan Meister', 21.65181529829283), ('Gerry Blair', 21.65181529829283), ('Matt Chamberlain', 21.65181529829283), ('Paul Yates', 21.65181529829283), ('Wray Wilson', 21.65181529829283), ('Yuri Bartoli', 21.65181529829283), ('Nick Bassett', 21.65181529829283), ('Rob Bavin'

In [27]:
import math
with open('movie_info.pkl','rb') as f:
    movie_info=pickle.load(f)
    
with open('name_url.pkl','rb') as f:
    movie_budget=pickle.load(f)
    
def get_company_info_helper(name, movie_info=movie_info, movie_budget=movie_budget):
    
    company_name_info=dict()
    cast_dic=movie_info[name]['summary']
    if name in movie_budget.keys() and movie_budget[name]['Release Date']<datetime.now():
        if 'Production Companies' in cast_dic.keys():
            for company in cast_dic['Production Companies']:
                #print(name, company.strip())
                bo=movie_budget[name]['Worldwide Gross']
                company_name_info[company.strip()]=company_name_info.get(company.strip(),[])+[bo]
    
    return company_name_info

In [91]:
def get_all_company_info():
    with open('movie_info.pkl', 'rb') as f:
        movie_info=pickle.load(f)
    
    total_dic=dict()
    keys=list(movie_info.keys())
    pool = Pool(processes=10)
    pool_outputs = pool.map(get_company_info_helper,keys)
    pool.close()
    pool.join()
    
    for item in pool_outputs:
        for key, value in item.items():
            total_dic[key.strip()]=total_dic.get(key.strip(),[])+value
    
    #pprint(total_dic)
    company_info=dict()
    for key,value in total_dic.items():
        company_info[key]=math.log(np.sum(np.array(value))/(len(value)+0.1)+1)
    
    with open('company_info.pkl','wb') as f:
        pickle.dump(company_info, f)
        
    return    

In [92]:
get_all_company_info()

In [93]:
import operator
with open('company_info.pkl', 'rb') as f:
    company_info=pickle.load(f)
    
sorted_company=sorted(company_info.items(), key=lambda d: d[1])
print(sorted_company[-30:])
print(sorted_company[:30])

[('Fairview Entertainment', 20.16510458271864), ('Kevin Feige', 20.188834252435928), ('Seven Bucks Productions', 20.196674530670407), ('DC Films', 20.206290199799334), ('John Calley', 20.211405000627998), ('Marvel Studios', 20.222776876389098), ('Steve Tisch Productions', 20.2420735023506), ('Esperanto Filmoj', 20.265466411578196), ('Ingenious Film Partners', 20.267055609994152), ('KatzSmith Productions', 20.267644437243806), ('Disney-Pixar', 20.281660856290824), ('Laura Ziskin Productions', 20.301337701538994), ('Tencent Pictures', 20.30207090709969), ('One Race Films', 20.320634559484038), ('Chris Meledandri', 20.326245639902215), ('Lucasfilm', 20.32877095859611), ('Simon Kinberg', 20.337419884892842), ('Wingnut Films', 20.346044102071257), ('Heyday Films', 20.35786380039569), ('Illumination Entertainment', 20.386172563069625), ('China Film Company', 20.409429556800788), ('Wanda Pictures', 20.42585515967782), ('Syncopy', 20.460710614331887), ('Fuji Television Network', 20.50784828028

In [ ]:
temp_list=get_roles()
def convert_helper(name, product_list=temp_list):
    print(name)
    total_dic=dict()
    with open('movie_info.pkl', 'rb') as f:
        movie_info=pickle.load(f)
    
    info=movie_info[name]
    total_dic['name']=name
    sum_list=['Franchise', 'Genre', 'Creative Type', 'Budget', 'Keywords', 'Production Companies','Production Countries', 'Production Method', 'Video Release']
    for cate in sum_list:
        cate_name='_'.join(cate.lower().split())
        if cate in info['summary'].keys():
            total_dic[cate_name]=info['summary'][cate]
        else:
            total_dic[cate_name]='-'
    
    for item in info['box_office']['demostic']:
        if 'Demostic' in item['Record']:
            total_dic['demostic']=item['Revenue']
            break
    
    international_list=['International', 'Worldwide']
    for cat in international_list:
        for item in info['international'][cat+' Box Office']:
            if cat in item['Record']:
                total_dic[cat.lower()]=item['Revenue']
                break
                
        if cat.lower() not in total_dic.keys():
            total_dic[cat.lower()]='-'
            
    cast_list=['cameos', 'supporting_cast', 'leading_members', 'supporting_cast']
    for cate in cast_list:
        if cate in info['cast'].keys():
            total_dic[cate]=[a['name'] for a in info['cast'][cate]]
        else:
            total_dic[cate]=[]
    
    if 'production' in info['cast'].keys():
        item=info['cast']['production']
        for role in product_list:
            total_dic[role]=[a['name'] for a in item if a['role']==role]
            
    return total_dic

In [ ]:
#convert the dictionary to pandas
total_dic=dict()
with open('name_url.pkl','rb') as f:
    name_dic=pickle.load(f)
    
keys= list(name_dic.keys())
pool = Pool(processes=10) 
pool_outputs = pool.map(convert_helper,keys)
pool.close()
pool.join()

for item in pool_outputs:
    for key, value in item.items():
        total_dic[key]=total_dic.get(key,[])+[value]
    
pdf=pd.DataFrame(data=total_dic, columns=total_dic.keys())
print(pdf) 

## Feature Selection


In [ ]:
#The importance of the feaures


In [21]:
#Select the important features
from tqdm import tqdm
for i in tqdm(range(10)):
    pass

100%|██████████| 10/10 [00:00<00:00, 6373.35it/s]


## Visualize the network


In [5]:
import networkx as nx

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx

G=nx.karate_club_graph()

plot = figure(title="Networkx Integration Demonstration", x_range=(-1.1,1.1), y_range=(-1.1,1.1))

graph = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))
plot.renderers.append(graph)

output_notebook()
show(plot)

Loading BokehJS ...